In [14]:
import numpy as np
import pandas as pd
import xarray as xr
import os
from matplotlib import pyplot as plt
import cartopy.crs as ccrs
import math as mt
import dask

In [2]:
M = 0.8 #change later
reference_fuel_consumption = 0.7296 #who knows
reference_nox = 19.1835 #IDGAF
number_of_engines = 2 
temp_sea_level = 288.15 #kelvins
pres_sea_level = 101325 #pascals
gamma = 1.4
R = 287.05


In [17]:
#merge echam
directory = os.fsencode("C:/Users/macie/OneDrive/Desktop/Project_A02/Project_data/DT00/AT20_optimal")
dir = "C:/Users/macie/OneDrive/Desktop/Project_A02/Project_data/DT00/AT20_optimal"
echam_files = []
for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if filename.endswith("ECHAM5.nc"):
        echam_files.append(os.path.join(dir, filename))
    else:
        continue
if echam_files:
    merged_echam = xr.open_mfdataset(echam_files, combine='nested', concat_dim='time')
    
#merge accf
directory = os.fsencode("C:/Users/macie/OneDrive/Desktop/Project_A02/Project_data/DT00/AT20_optimal")
dir = "C:/Users/macie/OneDrive/Desktop/Project_A02/Project_data/DT00/AT20_optimal"
accf_files = []
for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if filename.endswith("accf_gp.nc"):
        accf_files.append(os.path.join(dir, filename))
    else:
        continue
if accf_files:
    merged_accf = xr.open_mfdataset(accf_files, combine='nested', concat_dim='time')

#merge contrail
directory = os.fsencode("C:/Users/macie/OneDrive/Desktop/Project_A02/Project_data/DT00/AT20_optimal")
dir = "C:/Users/macie/OneDrive/Desktop/Project_A02/Project_data/DT00/AT20_optimal"
contrail_files = []
for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if filename.endswith("contrail_gp.nc"):
        contrail_files.append(os.path.join(dir, filename))
    else:
        continue
if contrail_files:
    merged_contrail = xr.open_mfdataset(contrail_files, combine='nested', concat_dim='time')

In [ ]:
#change the datasets so it correspomds with above code

#aircrafts velocity based on its mach number
velocity = M*np.sqrt(gamma*R*merged_echam['tm1'])
#correction coefficients for fuel consumption
delta = merged_echam['tm1']*(1+0.2*M**2)**3.5 / temp_sea_level
theta = merged_echam['press']*(1+0.2*M**2)**3.5 / pres_sea_level
#actual fuel consumption per engine
f_a_i = reference_fuel_consumption*delta*np.sqrt(theta)
#total fuel consumption
f_cr = f_a_i*number_of_engines
#humidity correction coefficient
H_c = np.exp(-19*(merged_echam['qm1']-0.00634))
#nox emission corrected for cruise conditions
EINO_x = reference_nox*delta**0.4*theta**0.3*H_c
#F-ATR20 for each emission in kelvins per kilometer
co2_atr20_perkm = merged_accf['atr20_co2']/velocity*f_cr
h2o_atr20_perkm = merged_accf['atr20_h2o']/velocity*f_cr
o3_atr20_perkm = merged_accf['atr20_o3']/velocity*f_cr*EINO_x*10**(-3)
ch4_atr20_perkm = merged_accf['atr20_ch4']/velocity*f_cr*EINO_x*10**(-3)
contr_atr20_perkm = merged_accf['atr20_contrail']*merged_contrail['potcov']

